# Laboratorio 2: Data Understanding

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Machine Learning Operations** 

## Integrantes

- Arturo Argueta - 21527 
- Edwin de León - 22809 
- Diego Leiva - 21752 
- Pablo Orellana - 21970

## Librerías

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from typing import Dict, Tuple
import logging

## Logging

In [ ]:
# Basic logging configuration
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more detailed output
    format="[%(levelname)s] - %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

## Lectura de datos

Durante el análisis exploratorio se encontró que hay conjuntos de datos con codificaciones diferentes a la típica utf-8, por lo que se necesita una lectura segura de datos

In [ ]:
def read_csv_with_fallback(
        path: Path, 
        encodings: Tuple[str, ...] = ("utf-8", "latin1", "cp1252"), 
        **pd_kwargs
    ) -> Tuple[pd.DataFrame, str]:
    """
    Attempts to read a CSV file with various encodings to find the correct one.

    Args:
        path (str): Path to the CSV file.
        encodings (tuple): Encodings to try.

    Returns:
        Tuple[pd.DataFrame, str]: The DataFrame read and the encoding used.
    """
    last_error = None

    # Iterating over encodings
    for encoding in encodings:
        try:
            # Try reading the CSV with the current encoding
            df = pd.read_csv(path, encoding=encoding, **pd_kwargs)
            # If successful, return the DataFrame and the encoding used
            return df, encoding

        # If it fails, try the next encoding
        except UnicodeDecodeError as e:
            last_error = e

        # If it fails, capture the error
        except Exception as e:
            # Other errors; keep track and keep trying in case it's just encoding
            last_error = e

    # Last "tolerant" attempt: utf-8 with replacement for bad characters
    try:
        with open(path, "r", encoding="utf-8", errors="replace") as f:
            df = pd.read_csv(f, **pd_kwargs)
            # If successful, return the DataFrame and the encoding used
            return df, "utf-8(errors=replace)"
    
    # If not successful, keep track of the last error
    except Exception:
        raise last_error

In [ ]:
def load_csvs(
        folder: str,
        pattern: str = "*.csv",
        encodings: Tuple[str, ...] = ("utf-8", "latin1", "cp1252"),
        **pd_kwargs
    ) -> Dict[str, pd.DataFrame]:
    """
    Load CSV files from a folder into a dictionary of DataFrames.
    
    Args:
        folder (str): The folder containing the CSV files.
        pattern (str, optional): The glob pattern to match CSV files. Defaults to "*.csv".
        encodings (Tuple[str, ...], optional): The encodings to try when reading CSV files. Defaults to ("utf-8", "latin1", "cp1252").
        **pd_kwargs: Additional keyword arguments to pass to pandas read_csv.

    Returns:
        Dict[str, pd.DataFrame]: A dictionary mapping file names (without extensions) to DataFrames.
    """
    # Get a list of all CSV files in the folder
    files = sorted(Path(folder).glob(pattern))
    dfs: Dict[str, pd.DataFrame] = {}

    # Read each CSV file into a DataFrame
    for file in tqdm(files, desc="Reading CSVs", unit="file"):
        df, encoding = read_csv_with_fallback(file, encodings=encodings, **pd_kwargs)
        dfs[file.stem] = df # Store the DataFrame with the file name (without extension) as the key

    return dfs

## Limpieza

### Manejo de duplicados

Durante el análisis exploratorio se encontró que los conjuntos de datos de clientes y eventos tienen registros completos duplicados.

In [ ]:
def duplicates_handling(input_dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Handle duplicates in a DataFrame.
    Args:
        data (pd.DataFrame): The DataFrame to process.

    Returns:
        pd.DataFrame: The DataFrame without duplicates.
    """
    df_unique = input_dataframe.drop_duplicates()  # Delete duplicates

    return df_unique

### Manejo de nulos

Durante el análisis exploratorio se encontró que el conjunto de datos de  `clientes` tiene 281 registros nulos en cada columna (2.34%). en `eventos` solo la variable `transactionid` tiene 99.9% de valores nulos, y para `producto` las variables `categoria_id` y `marca_id` tienen 8.55% y 7.28% respectivamente y `precio` tiene un 0.05% de nulos.

Para tratarlos se decide eliminar los resgistros con datos nulos para `clientes` y la variable `precio`, para las variables `categoria_id` y `marca_id` se decide mapearlos como desconocido u otros. 

In [ ]:
def next_free_id(series: pd.Series) -> int:
    """
    Get the next free (unused) ID from a series of existing IDs.

    Args:
        series (pd.Series): Series of existing IDs.

    Returns:
        int: Next free ID.
    """
    # Get the next free ID
    if series.empty:
        # If the series is empty, return 1 as the next ID
        return 1
    
    return int(pd.to_numeric(series, errors="coerce").max()) + 1

In [ ]:
def nulls_handling(dataframes: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Handle null values in the given DataFrames.
    
    Args:
        dataframes (Dict[str, pd.DataFrame]): The DataFrames to process.
    
    Returns:
        Dict[str, pd.DataFrame]: The processed DataFrames.
    """
    out: Dict[str, pd.DataFrame] = {} # Initialize output dictionary

    # Handle CATEGORIA
    category_df = dataframes["categoria"].copy() # create a copy of the categoria DataFrame
    
    # Check if 'Otro' category exists
    if not (category_df["categoria"] == "Otro").any():
        # If not, create it
        new_category_id = next_free_id(category_df["id"])
        # Append the new category with the next free ID and the name 'Otro'
        category_df = pd.concat(
            [category_df, pd.DataFrame([{"id": new_category_id, "categoria": "Otro"}])],
            ignore_index=True
        )
    out["categoria"] = category_df

    # Handle MARCA
    brand_df = dataframes["marca"].copy() # create a copy of the marca DataFrame

    # Check if 'Otro' brand exists
    if not (brand_df["marca"] == "Otro").any():
        # If not, create it
        new_brand_id = next_free_id(brand_df["id"])
        # Append the new brand with the next free ID and the name 'Otro'
        brand_df = pd.concat(
            [brand_df, pd.DataFrame([{"id": new_brand_id, "marca": "Otro"}])],
            ignore_index=True
        )
    out["marca"] = brand_df

    # Handle CLIENTE
    out["cliente"] = dataframes["cliente"].dropna(how="any") # Drop rows with any null values

    # Handle EVENTO
    event_df = dataframes["evento"].copy() # create a copy of the evento DataFrame
    
    # Check if 'transactionid' column exists
    if "transactionid" in event_df.columns:
        # Drop the 'transactionid' column
        event_df = event_df.drop(columns=["transactionid"])
    out["evento"] = event_df

    # Handle PRODUCTO
    product_df = dataframes["producto"].copy()
    product_df = product_df.dropna(subset=["precio"]) # Drop rows with null values in 'precio'

    # Check if 'categoria_id' column exists
    if "categoria_id" in product_df.columns:
        # Fill null values in 'categoria_id' with the ID of 'Otro' category
        unknown_category_id = category_df.loc[category_df["categoria"] == "Otro", "id"].iloc[0]
        product_df["categoria_id"] = product_df["categoria_id"].fillna(unknown_category_id).astype("Int64")

    # Check if 'marca_id' column exists
    if "marca_id" in product_df.columns:
        # Fill null values in 'marca_id' with the ID of 'Otro' brand
        unknown_brand_id = brand_df.loc[brand_df["marca"] == "Otro", "id"].iloc[0]
        product_df["marca_id"] = product_df["marca_id"].fillna(unknown_brand_id).astype("Int64")
    out["producto"] = product_df

    return out

### Manejo de formatos de fechas

In [ ]:
def convert_timestamp(
        input_dataframe: pd.DataFrame, 
        timestamp: str = "timestamp", 
        date: str = "fecha"
    ) -> pd.DataFrame:
    """
    Convert timestamp column to datetime.
    
    Args:
        data (pd.DataFrame): DataFrame to process.
        timestamp (str): Name of the timestamp column.
        date (str): Name of the resulting date column.

    Returns:
        pd.DataFrame: DataFrame with the converted date column.
    """
    sample_val = input_dataframe[timestamp].iloc[0] # Get a sample value from the timestamp column
    unit = 's' if sample_val < 1e11 else 'ms' # Determine the unit of the timestamp
    input_dataframe[date] = pd.to_datetime(input_dataframe[timestamp], unit=unit) # Convert the timestamp column to datetime
    return input_dataframe

In [ ]:
def clean_birthdate_and_age(
        dataframes: Dict[str, pd.DataFrame], 
        key_data: str = "cliente", 
        birth_date: str = "nacimiento", 
        client_age: str = "edad"
    ) -> Dict[str, pd.DataFrame]:
    """
    Clean birthdate and age fields in the specified DataFrame.
    Handle parsing, future date correction, and age recalculation.

    Args:
        dataframes (Dict[str, pd.DataFrame]): Dictionary of DataFrames.
        key_data (str): Key for the DataFrame to process.
        birth_date (str): Name of the birthdate column.
        client_age (str): Name of the age column.

    Returns:
        Dict[str, pd.DataFrame]: Updated dictionary of DataFrames.
    """
    df = dataframes[key_data].copy() # Create a copy of the DataFrame

    # Parse date field to datetime format
    df[birth_date] = pd.to_datetime(df[birth_date], format="%m/%d/%y", errors="coerce")

    # Correct future years
    future_date_mask = df[birth_date] > pd.Timestamp.today() # Identify future dates
    df.loc[future_date_mask, birth_date] -= pd.DateOffset(years=100) # Subtract 100 years from future dates

    # Recalculate age
    today = pd.Timestamp.today() # Get the current date
    df[client_age] = ((today - df[birth_date]).dt.days / 365.25).round() # Recalculate age

    dataframes[key_data] = df # Update the DataFrame in the dictionary

    return dataframes

## Transformación

### Integridad referencial y enriquecimiento

#### Filtro de productos válidos

Se aplica un filtro obligatorio para conservar únicamente aquellos eventos cuyo `itemid` esté presente y referenciado en `producto.id`. De esta forma se eliminan todos los registros donde el producto del evento es desconocido, garantizando la integridad referencial con la tabla de productos y permitiendo calcular métricas consistentes de ventas, categorías y marcas.

In [ ]:
def coerce_id_formats(
        event_dataframe: pd.DataFrame,
        product_dataframe: pd.DataFrame,
        col_itemid: str = "itemid",
        col_prod_id: str = "id"
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Coerce ID types for events and products DataFrames.

    Args:
        event_dataframe (pd.DataFrame): The events DataFrame.
        product_dataframe (pd.DataFrame): The products DataFrame.
        col_itemid (str): The column name for item IDs in events.
        col_prod_id (str): The column name for product IDs in products.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: The coerced events and products DataFrames.
    """
    # Create copies of the original DataFrames
    event_df = event_dataframe.copy()
    product_df = product_dataframe.copy()

    # Normalize ID types to Int64 (allows NA) for clean comparisons
    event_df[col_itemid] = pd.to_numeric(event_df[col_itemid], errors="coerce").astype("Int64")
    product_df[col_prod_id] = pd.to_numeric(product_df[col_prod_id], errors="coerce").astype("Int64")

    return event_df, product_df

In [ ]:
def filter_events_by_valid_product(
        event_dataframe: pd.DataFrame,
        product_dataframe: pd.DataFrame,
        col_itemid: str = "itemid",
        col_prod_id: str = "id"
    ) -> pd.DataFrame:
    """
    Filter events by valid product IDs.

    Args:
        event_dataframe (pd.DataFrame): DataFrame de eventos.
        product_dataframe (pd.DataFrame): DataFrame de productos.
        col_itemid (str): Nombre de la columna de itemid en el DataFrame de eventos.
        col_prod_id (str): Nombre de la columna de id en el DataFrame de productos.

    Returns:
        pd.DataFrame: DataFrame de eventos filtrados.
    """
    # Drop NA values from product IDs
    prod_ids = product_dataframe[col_prod_id].dropna().unique()

    # Generate mask for valid events
    mask_valid = event_dataframe[col_itemid].isin(prod_ids)
    valid_events = event_dataframe.loc[mask_valid].copy()

    # Sanity check: all item IDs in valid events must be in product IDs
    assert valid_events[col_itemid].dropna().isin(prod_ids).all(), \
        "Quedaron itemid no válidos tras el filtro."

    return valid_events

### Enriquecimiento de evento con datos de producto

Se integran los valores relacionados a la clave foránea de productos en la tabla de eventos, para enriquecer las visualizaciones posteriores con datos de los productos seleccionados, en lugar de usar el id del producto.

In [ ]:
def select_product_features(
        product_dataframe: pd.DataFrame,
        category_dataframe: pd.DataFrame,
        brand_dataframe: pd.DataFrame
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Select and rename relevant features from product, category, and brand dataframes.

    Args:
        product_dataframe (pd.DataFrame): The product dataframe.
        category_dataframe (pd.DataFrame): The category dataframe.
        brand_dataframe (pd.DataFrame): The brand dataframe.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: The selected and renamed dataframes.
    """

    # Select relevant columns from product dataframe
    selected_product_data = product_dataframe[
        ["id", "categoria_id", "nombre", "marca_id", "volumen", "precio"]].copy()

    # Rename product columns
    selected_product_data = selected_product_data.rename(
        columns={"nombre": "producto"})

    # Rename category columns
    selected_category_data = category_dataframe.rename(
        columns={"id": "categoria_id", "categoria": "categoria"}).copy()

    # Rename brand columns
    selected_brand_data = brand_dataframe.rename(
        columns={"id": "marca_id", "marca": "marca"}).copy()

    return selected_product_data, selected_category_data, selected_brand_data

In [ ]:
def enrich_with_product(
        event_dataframe: pd.DataFrame,
        product_dataframe: pd.DataFrame,
        col_itemid: str = "itemid",
        col_prod_id: str = "id"
    ) -> pd.DataFrame:
    """
    Enrich the event dataframe with product information,
    by merging with the product dataframe.

    Args:
        event_dataframe (pd.DataFrame): The event dataframe.
        product_dataframe (pd.DataFrame): The product dataframe.
        col_itemid (str): The column name for item id in the event dataframe.
        col_prod_id (str): The column name for product id in the product dataframe.

    Returns:
        pd.DataFrame: The enriched event dataframe.
    """
    # Merge event and product dataframes where product id matches
    merged_dataframe = event_dataframe.merge(
        product_dataframe,
        left_on=col_itemid,
        right_on=col_prod_id,
        how="left"
    )

    merged_dataframe = merged_dataframe.drop(columns=[col_prod_id])  # Drop duplicate product id
    
    return merged_dataframe

In [ ]:
def enrich_with_category_and_brand(
        event_dataframe: pd.DataFrame,
        category_dataframe: pd.DataFrame,
        brand_dataframe: pd.DataFrame
    ) -> pd.DataFrame:
    """
    Enrich event data with category and brand information.

    Args:
        event_dataframe (pd.DataFrame): The event DataFrame to enrich.
        category_dataframe (pd.DataFrame): The category DataFrame.
        brand_dataframe (pd.DataFrame): The brand DataFrame.

    Returns:
        pd.DataFrame: The enriched event DataFrame.
    """
    # Merge category information
    df = event_dataframe.merge(category_dataframe, on="categoria_id", how="left")
    # Merge brand information
    df = df.merge(brand_dataframe, on="marca_id", how="left")
    # Drop internal IDs that do not contribute
    df = df.drop(columns=["categoria_id", "marca_id"])

    return df

### Enriquecimiento de evento con datos de cliente

Se integran a la tabla de eventos las características relevantes de cada cliente a partir de la clave foránea `visitorid`, conservando únicamente los campos útiles para el análisis: nombre completo (combinación de nombre y apellido), género y edad previamente calculada. Para los registros sin coincidencia en la tabla de clientes se asigna la etiqueta `"Anónimo"` y se marca el indicador `cliente_conocido` como `False`, lo que permite diferenciar entre clientes identificados y no identificados. Este enfoque facilita segmentar métricas en el dashboard sin perder volumen de datos en los indicadores globales. 

**Nota**: a columna `ciudad` fue omitida debido a problemas de codificación (mojibake y mangled encoding) que no pudieron ser corregidos de forma confiable, evitando así mostrar valores ilegibles en el dashboard.

In [ ]:
def select_client_features(client_dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Select relevant columns for client features.

    Args:
        client_dataframe (pd.DataFrame): The client DataFrame to process.

    Returns:
        pd.DataFrame: The DataFrame with selected client features.
    """
    # Select relevant columns for client features
    cli_sel = client_dataframe[["id", "nombre", "apellido", "genero", "edad"]].copy()
    
    return cli_sel

In [ ]:
def enrich_with_client(
        event_dataframe: pd.DataFrame,
        client_dataframe: pd.DataFrame,
        col_visitorid: str = "visitorid",
        col_clientid: str = "id"
    ) -> pd.DataFrame:
    """
    Enrich event data with client information by merging on visitor ID when possible.

    Args:
        event_dataframe (pd.DataFrame): The event DataFrame to enrich.
        client_dataframe (pd.DataFrame): The client DataFrame to merge with.
        col_visitorid (str): The column name for visitor ID in the event DataFrame.
        col_clientid (str): The column name for client ID in the client DataFrame.

    Returns:
        pd.DataFrame: The enriched event DataFrame.
    """
    # Merge client information with event data
    df = event_dataframe.merge(client_dataframe, left_on=col_visitorid, right_on=col_clientid, how="left")

    # Flag `cliente_conocido` to indicate known clients
    df["cliente_conocido"] = df[col_clientid].notna()

    # Combine first and last name of client
    df["cliente"] = (df["nombre"].fillna("") + " " + df["apellido"].fillna("")).str.strip()
    df.loc[~df["cliente_conocido"], "cliente"] = "Anónimo" # Set anonymous for unknown clients

    # Fill missing values of genre and age
    df["genero"] = df["genero"].fillna("NA")
    df["edad"] = df["edad"]  # Fill missing values with NA

    # Drop unnecessary columns
    df = df.drop(columns=["nombre", "apellido", col_clientid])

    return df

### Selección de Variables de interés

En esta etapa se depura el dataset resultante para conservar únicamente las columnas necesarias para el dashboard ejecutivo, eliminando campos internos o no relevantes para el análisis. Se renombra la columna `event` a `evento` para mayor claridad, y se mantiene la siguiente información: `evento`, `fecha`, `producto`, `volumen`, `precio`, `categoria`, `marca`, `cliente`, `genero`, `edad` y el indicador `cliente_conocido`. Este formato simplificado facilita la integración con Power BI y optimiza el rendimiento en la visualización, evitando la carga de datos innecesarios.

In [ ]:
def feature_selection(input_dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Select relevant features for the dashboard.

    Args:
        input_dataframe (pd.DataFrame): The input DataFrame to select features from.

    Returns:
        pd.DataFrame: The DataFrame with selected features.
    """
    # Define final features for the dataframe
    selected_columns = [
        "event",
        "fecha",
        "producto",
        "volumen",
        "precio",
        "categoria",
        "marca",
        "cliente",
        "genero",
        "edad",
        "cliente_conocido"
    ]

    # Filter existing columns
    cols_existentes = [c for c in selected_columns if c in input_dataframe.columns]
    df_final = input_dataframe[cols_existentes].copy() # Make a copy
    df_final = df_final.rename(columns={"event": "evento"}) # Rename event to evento
    
    return df_final

## Pipeline Integrado

In [ ]:
def run_transform_pipeline(
        raw_folder: str = "data/raw",
        save_path: str = "data/processed/boozemart_data.csv"
    ) -> pd.DataFrame:
    """
    Run the data transformation pipeline to process and clean the raw data.
    Steps:
    1) Load csv files
    2) Handle duplicates
    3) Handle null values
    4) Convert timestamps to datetime
    5) Process birth date and age of client
    6) Coerce IDs and filter events by valid product relationships
    7) Enrich event dataframe with product, brand and category information
    8) Enrich event dataframe with client relevant data
    9) Feature selection
    10) Save the processed data into a csv file

    Args:
        raw_folder (str): Path to the folder containing raw data files.
        save_path (str): Path to the folder where the processed data will be saved.

    Returns:
        pd.DataFrame: The processed and cleaned data.
    """
    logger.info("Starting data transformation pipeline...")
    
    # 1) Load csv files
    logger.info("Loading CSV files from %s", raw_folder)
    dfs = load_csvs(raw_folder, sep=",")

    # 2) Handle duplicates
    logger.info("Handling duplicates in dataframes...")
    dfs["cliente"] = duplicates_handling(dfs["cliente"])
    dfs["evento"]  = duplicates_handling(dfs["evento"])

    # 3) Handle null values
    logger.info("Handling null values in dataframes...")
    dfs_limpio = nulls_handling(dfs)

    # 4) Convert timestamps to datetime
    logger.info("Converting timestamps to datetime...")
    dfs_limpio["evento"] = convert_timestamp(dfs_limpio["evento"], timestamp="timestamp")

    # 5) Process birth date and age of client
    logger.info("Cleaning birthdate and age in client dataframe...")
    dfs_limpio = clean_birthdate_and_age(dfs_limpio, key_data="cliente")

    # 6) Coerce IDs and filter events by valid product relationships
    logger.info("Coercing ID formats and filtering events by valid products...")
    evento_t, producto_t = coerce_id_formats(dfs_limpio["evento"], dfs_limpio["producto"])
    valid_events = filter_events_by_valid_product(evento_t, producto_t)

    # 7) Enrich event dataframe with product, brand and category information
    logger.info("Enriching event dataframe with product, category, and brand information...")
    sel_prod, sel_cat, sel_brand = select_product_features(
        dfs_limpio["producto"], dfs_limpio["categoria"], dfs_limpio["marca"]
    )
    evento_prod = enrich_with_product(valid_events, sel_prod)
    evento_full = enrich_with_category_and_brand(evento_prod, sel_cat, sel_brand)

    # 8) Enrich event dataframe with client relevant data
    logger.info("Enriching event dataframe with client information...")
    cli_sel = select_client_features(dfs_limpio["cliente"])
    evento_cli = enrich_with_client(evento_full, cli_sel)

    # 9) Feature selection
    logger.info("Selecting relevant features for the final dataframe...")
    final_df = feature_selection(evento_cli)

    # 10) Save the processed data into a csv file
    if save_path:
        logger.info("Saving processed data to %s", save_path)
        final_df.to_csv(save_path, index=False, compression="gzip")

    return final_df

## Ejecución

In [ ]:
boozemart_data = run_transform_pipeline()

In [ ]:
boozemart_data.head()